In [12]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import pandas as pd
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as SIA

def WebScraper(business, location, driver_path=r'.\chromedriver-win64\chromedriver.exe'):
  
    driver = webdriver.Chrome(r'./chromedriver-win64/chromedriver.exe')

    driver.get("https://www.google.com")
    
    # Search for the business name and location
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(f"{business} {location} reviews")
    search_box.send_keys(Keys.RETURN)
    time.sleep(1.5)
    
    # Wait for the review button to be clickable
    try:
        review_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-async-trigger='reviewDialog']"))
        )
        
        # Get the number of reviews
        review_text = review_button.text
        review_counts_str = review_text.split(" Google reviews")[0].replace(",", "")
        review_counts = int(review_counts_str)
        
        # Open the reviews modal
        review_button.click()
        time.sleep(1)  # Allow modal to open
        
        avg_rating = driver.find_element(By.CLASS_NAME, "Aq14fc").text.strip()
        
        # Wait for the "Newest" filter to be clickable and click it
        newest_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@aria-checked='false']//span[text()='Newest']"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", newest_button)
        driver.execute_script("arguments[0].click();", newest_button)

        review_limit = min(review_counts, 25)  # Limit to 100 reviews
        review_count = 0

        # Scroll and scrape reviews
        reviews_list = []
        date_list = []
        
        while review_count < review_limit:
                    # Wait for the review containers to appear
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "BgXiYe"))
                )

                    # Re-fetch the review containers after scrolling
            review_containers = driver.find_elements(By.CLASS_NAME, "BgXiYe")
            scrollable_element = driver.find_element(By.CLASS_NAME, "review-dialog-list")
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)

                    # Iterate through review containers and extract the data
            for container in review_containers:
                container_soup = BeautifulSoup(container.get_attribute('outerHTML'), 'html.parser')

                date_span = container_soup.find('span', class_="dehysf lTi8oc")
                date = date_span.get_text(strip=True) if date_span else None
    
                full_text_span = container_soup.find('span', class_='review-full-text')
                if full_text_span and full_text_span.get_text(strip=True):
                    text = full_text_span.get_text(strip=True)
                else:
                    alternative_text_span = container_soup.find('span', attrs={'data-expandable-section': True})
                    text = alternative_text_span.get_text(strip=True) if alternative_text_span else None

                if text and text.strip():
                    if text not in reviews_list:
                        reviews_list.append(text)
                        date_list.append(date)
                        review_count += 1
                if review_count >= review_limit:
                    break
                   

        driver.quit()

    

        # Convert the lists into a DataFrame
        reviews_df = pd.DataFrame({
            "Review": reviews_list,
            "Date": date_list
        })

        return reviews_df, avg_rating
    
    except Exception as e:
        print("An error occurred:", e)
        driver.quit()
        return pd.DataFrame()  # Return empty DataFrame on error

def GetPolarity(review):
    analyser = SIA()
    return analyser.polarity_scores(review)['compound']

def GetSentiment(polarity):
    if polarity > 0.1:
        return "Positive"
    elif polarity < -0.65:
        return "Negative"
    else:
        return "Neutral"

def App(business_name, location_name):
    reviews_df, avg_rating = WebScraper(business_name, location_name) 
    
    # Apply sentiment analysis to each review
    reviews_df['polarity'] = reviews_df["Review"].apply(GetPolarity)  
    reviews_df['sentiment'] = reviews_df['polarity'].apply(GetSentiment)
    positive_dict = {}
    negative_dict = {}

    if (reviews_df['sentiment'] == 'Positive').sum() > 0:
        limit = min(5, (reviews_df['sentiment'] == 'Positive').sum())
        top_positive = reviews_df[reviews_df['sentiment'] == 'Positive'].nlargest(limit, 'polarity')[['Review', 'Date']]
        positive_dict = dict(zip(top_positive['Review'], top_positive['Date']))
    else:
        top_positive = "No positive comments from past 30 reviewers!"

    if (reviews_df['sentiment'] == 'Negative').sum() > 0:
        limit = min(5, (reviews_df['sentiment'] == 'Negative').sum())
        top_negative = reviews_df[reviews_df['sentiment'] == 'Negative'].nsmallest(limit, 'polarity')[['Review', 'Date']]
        negative_dict = dict((top_negative['Review'], top_negative['Date']))
    else:
        top_negative = "No Negative comments from past 30 reviewers!"
        
    return {'a'}

# Example Usage
a= time.time()
positive_reviews, negative_reviews = App("debonairs", "surulere")
print("Positive Reviews: ", positive_reviews)
print("Negative Reviews: ", negative_reviews)
b= time.time()
print(b-a)


IndentationError: unindent does not match any outer indentation level (<tokenize>, line 54)

In [79]:
from selenium import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd

def WebScraper(business, location, driver_path=r'.\chromedriver-win64\chromedriver.exe'):
    driver = webdriver.Chrome(driver_path)
    
    # Open Google search page
    driver.get("https://www.google.com")
    
    # Search for the business name and location
    search_box = driver.find_element(By.NAME, "q")
    search_box.send_keys(f"{business} {location} reviews")
    search_box.send_keys(Keys.RETURN)
    
    # Wait for the review button to be clickable
    try:
        review_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, "a[data-async-trigger='reviewDialog']"))
        )
        
        # Get the number of reviews
        review_text = review_button.text
        review_counts_str = review_text.split(" Google reviews")[0].replace(",", "")
        review_counts = int(review_counts_str)
        
        # Open the reviews modal
        review_button.click()
        WebDriverWait(driver, 3)  # Wait for modal to open
        
        # Get the average rating
        avg_rating = driver.find_element(By.CLASS_NAME, "Aq14fc").text.strip()
        
        # Wait for the "Newest" filter to be clickable and click it
        newest_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, "//div[@aria-checked='false']//span[text()='Newest']"))
        )
        driver.execute_script("arguments[0].scrollIntoView(true);", newest_button)
        driver.execute_script("arguments[0].click();", newest_button)
        time.sleep(1.5)
        review_limit = min(review_counts, 25)  # Limit to 25 reviews
        review_count = 0

        reviews_list = []
        date_list = []
        
        while review_count < review_limit:
            # Wait for review containers to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "BgXiYe"))
            )
            
            # Re-fetch review containers after waiting for them to load
            review_containers = driver.find_elements(By.CLASS_NAME, "BgXiYe")
            
            # Scroll down to load new reviews
            scrollable_element = driver.find_element(By.CLASS_NAME, "review-dialog-list")
            driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", scrollable_element)

            # Wait for new reviews to load
            WebDriverWait(driver, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "BgXiYe"))
            )

            # Iterate through the review containers
            for container in review_containers:
                try:
                    container_soup = BeautifulSoup(container.get_attribute('outerHTML'), 'html.parser')

                    # Extract review date
                    date_span = container_soup.find('span', class_="dehysf lTi8oc")
                    date = date_span.get_text(strip=True) if date_span else None

                    # Extract review text
                    full_text_span = container_soup.find('span', class_='review-full-text')
                    if full_text_span and full_text_span.get_text(strip=True):
                        text = full_text_span.get_text(strip=True)
                    else:
                        alternative_text_span = container_soup.find('span', attrs={'data-expandable-section': True})
                        text = alternative_text_span.get_text(strip=True) if alternative_text_span else None

                    # Add the review if it's new and not already collected
                    if text and text.strip() and text not in reviews_list:
                        reviews_list.append(text)
                        date_list.append(date)
                        review_count += 1

                    # Stop once the review count limit is reached
                    if review_count >= review_limit:
                        break
                except Exception as e:
                    print(f"Error while processing review: {e}")

            # Check if the review count exceeds the limit
            if review_count >= review_limit:
                break

        driver.quit()

        # Convert the lists into a DataFrame
        reviews_df = pd.DataFrame({
            "Review": reviews_list,
            "Date": date_list
        })

        return reviews_df, avg_rating

    except Exception as e:
        print(f"An error occurred: {e}")
        driver.quit()
        return pd.DataFrame(), None


In [80]:
Reviews_df,avg =WebScraper( "dominos pizza", "itire")

In [81]:
Reviews_df

,Review,Date
0,Food: 5/5|Service: 5/5|Atmosphere: 5/5Recommen...,3 weeks ago
1,AwesomeFood: 5/5|Service: 5/5|Atmosphere: 5/5,4 weeks ago
2,This is a good place that offers great food bu...,a month ago
3,Everything is nice 👍Food: 5/5|Service: 5/5|Atm...,a month ago
4,Food: 4/5|Service: 5/5|Atmosphere: 2/5,a month ago
5,These guys are thieves. I placed an order onli...,a month ago
6,"Chicken was undercooked, and overpricedFood: 1...",2 months ago
7,Placed an order through glovo over one hour ag...,2 months ago
8,Service: 2/5|Atmosphere: 3/5Recommended dishes...,2 months ago
9,"Very terrible service, placed an order and it ...",2 months ago
